In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ws37emcl
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ws37emcl
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=f78af9a8412d8f02a834c6136028cd918ca5757a585370301cd0e97aad926b43
  Stored in directory: /tmp/pip-ephem-wheel-cache-gebrey90/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# Código do exercício 1.

In [4]:
%%cu

#include <stdio.h>

void cpu_helloworld() {
    printf("Alô da CPU!\n");
}

__global__ void gpu_helloworld() {
    int threadId = threadIdx.x;
    printf("Alô da GPU! Meu threadId eh %d\n", threadId);
}

int main(int argc, char **argv) {
    dim3 grid(1);
    dim3 block(32);

    cpu_helloworld();

    gpu_helloworld<<<grid, block>>>();

    cudaDeviceSynchronize();

    return 0;
}

Alô da CPU!
Alô da GPU! Meu threadId eh 0
Alô da GPU! Meu threadId eh 1
Alô da GPU! Meu threadId eh 2
Alô da GPU! Meu threadId eh 3
Alô da GPU! Meu threadId eh 4
Alô da GPU! Meu threadId eh 5
Alô da GPU! Meu threadId eh 6
Alô da GPU! Meu threadId eh 7
Alô da GPU! Meu threadId eh 8
Alô da GPU! Meu threadId eh 9
Alô da GPU! Meu threadId eh 10
Alô da GPU! Meu threadId eh 11
Alô da GPU! Meu threadId eh 12
Alô da GPU! Meu threadId eh 13
Alô da GPU! Meu threadId eh 14
Alô da GPU! Meu threadId eh 15
Alô da GPU! Meu threadId eh 16
Alô da GPU! Meu threadId eh 17
Alô da GPU! Meu threadId eh 18
Alô da GPU! Meu threadId eh 19
Alô da GPU! Meu threadId eh 20
Alô da GPU! Meu threadId eh 21
Alô da GPU! Meu threadId eh 22
Alô da GPU! Meu threadId eh 23
Alô da GPU! Meu threadId eh 24
Alô da GPU! Meu threadId eh 25
Alô da GPU! Meu threadId eh 26
Alô da GPU! Meu threadId eh 27
Alô da GPU! Meu threadId eh 28
Alô da GPU! Meu threadId eh 29
Alô da GPU! Meu threadId eh 30
Alô da GPU! Meu threadId eh 31



# Código do exercício 2.

In [38]:
%%cu

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

#define BLOCK_SIZE 64
#define ARRAY_SIZE 128000

typedef struct timeval tval;

float generate_hash(int n, float *y) {
    float hash = 0.0f;

    for (int i = 0; i < n; i++) {
        hash += y[i];
    }

    return hash;
}

double get_elapsed(tval t0, tval t1) {
    return (double)(t1.tv_sec - t0.tv_sec) * 1000.0L +
           (double)(t1.tv_usec - t0.tv_usec) / 1000.0L;
}

void cpu_saxpy(int n, float a, float *x, float *y) {
    for (int i = 0; i < n; i++) {
        y[i] = a * x[i] + y[i];
    }
}

// TO-DO #2.1
// Declarando e implementando o kernel CUDA executará na GPU.
__global__ void gpu_saxpy(int n, float a, float *x, float *y) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < n) {
        y[tid] = a * x[tid] + y[tid];
    }
}

int main(int argc, char **argv) {
    float a = 0.0f;
    float *x = NULL;
    float *y = NULL;
    float error = 0.0f;

    // Verifique se a contante foi fornecida
    // if (argc != 2) {
    //     fprintf(stderr, "Erro: A constante está faltando!\n");
    //     return -1;
    // }

    // Código original do problema SAXPY
    a = 10;
    x = (float *)malloc(sizeof(float) * ARRAY_SIZE);
    y = (float *)malloc(sizeof(float) * ARRAY_SIZE);
    for (int i = 0; i < ARRAY_SIZE; i++) {
        x[i] = 0.1f;
        y[i] = 0.2f;
    }

    // TO-DO #2.2
    // Definindo a distribuição das threads, em termos da dimensão da grade
    // (grid) e da dimensão de cada bloco (de threads) dentro da grade.
    dim3 grid((ARRAY_SIZE + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 block(BLOCK_SIZE);

    // TO-DO #2.3.1
    // Declarando e definindo a memória GPU necessária para executar o kernel
    // CUDA.
    float *d_x, *d_y;
    cudaMalloc((void **)&d_x, sizeof(float) * ARRAY_SIZE);
    cudaMalloc((void **)&d_y, sizeof(float) * ARRAY_SIZE);

    // TO-DO #2.3.2
    // Transferindo os dados do host para a GPU.
    cudaMemcpy(d_x, x, sizeof(float) * ARRAY_SIZE, cudaMemcpyHostToDevice);
    cudaMemcpy(d_y, y, sizeof(float) * ARRAY_SIZE, cudaMemcpyHostToDevice);

    // Variáveis para métricas de performance/tempo de execução da CPU.
    tval t_start, t_end;
    gettimeofday(&t_start, NULL);
    // Execução na CPU para fins de comparação
    cpu_saxpy(ARRAY_SIZE, a, x, y);
    gettimeofday(&t_end, NULL);
    double elapsed_cpu = get_elapsed(t_start, t_end);
    error = generate_hash(ARRAY_SIZE, y);

    printf("Execução na CPU terminada. Taxa de erro = %.6f.\n", error);
    printf("Tempo de execução na CPU: %.6f ms.\n", elapsed_cpu);

    // Variáveis para métricas de performance/tempo de execução da GPU.
    tval t_start_gpu, t_end_gpu;
    gettimeofday(&t_start_gpu, NULL);

    // TO-DO #2.4
    // Execute o kernel CUDA com os parâmetros correspondentes.
    gpu_saxpy<<<grid, block>>>(ARRAY_SIZE, a, d_x, d_y);
    cudaDeviceSynchronize();

    gettimeofday(&t_end_gpu, NULL);
    double elapsed_gpu = get_elapsed(t_start_gpu, t_end_gpu);

    // TO-DO #2.5.1
    // Transferindo os resultados da GPU para o host.
    cudaMemcpy(y, d_y, sizeof(float) * ARRAY_SIZE, cudaMemcpyDeviceToHost);

    error = fabsf(error - generate_hash(ARRAY_SIZE, y));

    printf("Execução na GPU terminada. Taxa de erro = %.6f.\n", error);
    printf("Tempo de execução na GPU: %.6f ms.\n", elapsed_gpu);

    if (error > 0.0001f) {
        fprintf(stderr, "Erro: a solução está incorreta!\n");
    }

    // Gerenciamento de memória.
    free(x);
    free(y);
    cudaFree(d_x);
    cudaFree(d_y);

    return 0;
}

Execução na CPU terminada. Taxa de erro = 153799.109375.
Tempo de execução na CPU: 0.329000 ms.
Execução na GPU terminada. Taxa de erro = 0.000000.
Tempo de execução na GPU: 0.054000 ms

